In [2]:
! pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 57.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=642f8353f95a6eee16ff76e816495ccb0bb394cb46996602ba1eb9eff1816fbe
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [12]:
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [15]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.util import MLUtils

import pandas as pd 


# Load and parse the data file, converting it to a DataFrame.
data = spark.read.csv("weatherAUS.csv", inferSchema= True, header = True) 


In [16]:
data = data.drop("Date","Location","Evaporation","Sunshine","Cloud9am",'Cloud3pm','Temp3pm',"Temp9am","Temp3am")

In [23]:
# GET RID OF NA VALUES 

from pyspark.sql.functions import mean
from pyspark.sql import functions as F

for i in data.columns:
  count = data.filter(data[i]=='NA').count()

numeric_column = ['MinTemp','MaxTemp', 'Rainfall','WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm','Humidity9am','Humidity3pm','Pressure9am','Pressure3pm']

# calculate average
for i in numeric_column:
  average = data.select([mean(i)])

# substitute null values with the average
updatedmint = data.withColumn("MinTemp",F.when(F.col("MinTemp") == 'NA',12.2).otherwise(F.col("MinTemp")))
updatedmaxt = updatedmint.withColumn("MaxTemp",F.when(F.col("MaxTemp") == 'NA',23.2).otherwise(F.col("MaxTemp")))
updatedrainf = updatedmaxt.withColumn("Rainfall",F.when(F.col("Rainfall") == 'NA',2.3).otherwise(F.col("Rainfall")))
updatedwind = updatedrainf.withColumn("WindGustSpeed",F.when(F.col("WindGustSpeed") == 'NA',40.0).otherwise(F.col("WindGustSpeed")))
updatedwinds9 = updatedwind.withColumn("WindSpeed9am",F.when(F.col("WindSpeed9am") == 'NA',14.0).otherwise(F.col("WindSpeed9am")))
updatedwinds3 = updatedwinds9.withColumn("WindSpeed3pm",F.when(F.col("WindSpeed3pm") == 'NA',18.6).otherwise(F.col("WindSpeed3pm")))
updatedhum9 = updatedwinds3.withColumn("Humidity9am",F.when(F.col("Humidity9am") == 'NA',68.8).otherwise(F.col("Humidity9am")))
updatedhum3 = updatedhum9.withColumn("Humidity3pm",F.when(F.col("Humidity3pm") == 'NA',51.5).otherwise(F.col("Humidity3pm")))
updatedpres9 = updatedhum3.withColumn("Pressure9am",F.when(F.col("Pressure9am") == 'NA',1017.7).otherwise(F.col("Pressure9am")))


In [24]:
data.printSchema()

root
 |-- MinTemp: string (nullable = true)
 |-- MaxTemp: string (nullable = true)
 |-- Rainfall: string (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: string (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: string (nullable = true)
 |-- WindSpeed3pm: string (nullable = true)
 |-- Humidity9am: string (nullable = true)
 |-- Humidity3pm: string (nullable = true)
 |-- Pressure9am: string (nullable = true)
 |-- Pressure3pm: string (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RainTomorrow: string (nullable = true)



In [25]:
data.show()

+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+---------+------------+
|MinTemp|MaxTemp|Rainfall|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|RainToday|RainTomorrow|
+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+---------+------------+
|   13.4|   22.9|     0.6|          W|           44|         W|       WNW|          20|          24|         71|         22|     1007.7|     1007.1|       No|          No|
|    7.4|   25.1|       0|        WNW|           44|       NNW|       WSW|           4|          22|         44|         25|     1010.6|     1007.8|       No|          No|
|   12.9|   25.7|       0|        WSW|           46|         W|       WSW|          19|          26|         38|         30|     1007.6|    

In [26]:
data.distinct().show()

+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+---------+------------+
|MinTemp|MaxTemp|Rainfall|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|RainToday|RainTomorrow|
+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+---------+------------+
|    4.7|   12.2|       0|        NNW|           24|       ESE|         N|           2|          13|         99|         75|     1015.5|     1012.7|       No|         Yes|
|    1.6|   17.2|       0|          S|           20|       ESE|         S|           6|           7|         68|         44|     1022.5|     1019.1|       No|          No|
|   10.6|   24.7|       0|         NE|           33|        NA|       ENE|           0|          20|         75|         40|     1017.9|    

In [32]:
# tbh idk whats going on, so im just gonna drop more columns....
data = data.drop("MinTemp","MaxTemp","WindGustDir","WindGustSpeed","WindDir9am","WindDir3pm","WindSpeed9am","WindSpeed3pm","Humidity9am","Humidity3pm","Pressure9am","Pressure3pm")
data.distinct().show()

+--------+---------+------------+
|Rainfall|RainToday|RainTomorrow|
+--------+---------+------------+
|     1.4|      Yes|         Yes|
|     7.6|      Yes|         Yes|
|      41|      Yes|          No|
|    31.2|      Yes|         Yes|
|    45.5|      Yes|         Yes|
|      57|      Yes|         Yes|
|    46.6|      Yes|         Yes|
|     5.5|      Yes|         Yes|
|     5.2|      Yes|          NA|
|     0.8|       No|          NA|
|       6|      Yes|          NA|
|    92.2|      Yes|         Yes|
|     9.8|      Yes|          NA|
|      NA|       NA|          No|
|    12.6|      Yes|          No|
|    13.4|      Yes|          No|
|     5.2|      Yes|         Yes|
|     2.4|      Yes|          NA|
|     4.6|      Yes|          NA|
|    46.4|      Yes|          No|
+--------+---------+------------+
only showing top 20 rows



In [34]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCols=["Rainfall","RainToday","RainTomorrow"], 
                             outputCols=["rain_index","today_index","tomorrow_index"]).fit(data).transform(data)

In [35]:
# rain tomorrow index: yes = 1.0, no = 0.0, NA = 2.0
# rain today index: yes = 1.0, no = 0.0, NA = 2.0

labelIndexer.show()

+--------+---------+------------+----------+-----------+--------------+
|Rainfall|RainToday|RainTomorrow|rain_index|today_index|tomorrow_index|
+--------+---------+------------+----------+-----------+--------------+
|     0.6|       No|          No|       4.0|        0.0|           0.0|
|       0|       No|          No|       0.0|        0.0|           0.0|
|       0|       No|          No|       0.0|        0.0|           0.0|
|       0|       No|          No|       0.0|        0.0|           0.0|
|       1|       No|          No|       6.0|        0.0|           0.0|
|     0.2|       No|          No|       1.0|        0.0|           0.0|
|       0|       No|          No|       0.0|        0.0|           0.0|
|       0|       No|          No|       0.0|        0.0|           0.0|
|       0|       No|         Yes|       0.0|        0.0|           1.0|
|     1.4|      Yes|          No|       8.0|        1.0|           0.0|
|       0|       No|         Yes|       0.0|        0.0|        

In [49]:
categorical_columns = ['RainToday','RainTomorrow']

from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder(inputCols=["rain_index","today_index","tomorrow_index"],
                        outputCols=["rain_onehot","today_onehot","tomorrow_onehot"])
models = encoder.fit(labelIndexer)
encoded = models.transform(labelIndexer)


In [50]:
encoded.show()

+--------+---------+------------+----------+-----------+--------------+----------------+-------------+---------------+
|Rainfall|RainToday|RainTomorrow|rain_index|today_index|tomorrow_index|     rain_onehot| today_onehot|tomorrow_onehot|
+--------+---------+------------+----------+-----------+--------------+----------------+-------------+---------------+
|     0.6|       No|          No|       4.0|        0.0|           0.0| (681,[4],[1.0])|(2,[0],[1.0])|  (2,[0],[1.0])|
|       0|       No|          No|       0.0|        0.0|           0.0| (681,[0],[1.0])|(2,[0],[1.0])|  (2,[0],[1.0])|
|       0|       No|          No|       0.0|        0.0|           0.0| (681,[0],[1.0])|(2,[0],[1.0])|  (2,[0],[1.0])|
|       0|       No|          No|       0.0|        0.0|           0.0| (681,[0],[1.0])|(2,[0],[1.0])|  (2,[0],[1.0])|
|       1|       No|          No|       6.0|        0.0|           0.0| (681,[6],[1.0])|(2,[0],[1.0])|  (2,[0],[1.0])|
|     0.2|       No|          No|       1.0|    

In [39]:
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer = VectorIndexer(inputCol=["RainTomorrow"], outputCol=["tomorrow_index"], maxCategories=4).fit(data)


TypeError: ignored